In [23]:
import pennylane as qml
from pennylane import numpy as np
from typing import (List, Tuple)

#Step 1: Input the known plaintext and ciphertext

#if __name__ == "__main__":
    #pt = input("Enter 8 bit plaintext: ")
    #ct = input("Enter 8 bit ciphertext: ")

pt= "11111111"
ct= "00010100"
#key-0, pt-1:00010100 , pt-0:

In [24]:
#Step 2: Encode the ciphertext into a Hamiltonian

#ct="01100000" for all zeroes


def w(ct,i,j):
    if int(ct[i]) == int(ct[j]):
        return -1
    if  int(ct[i]) != int(ct[j]):
        return 1
    

def t(ct, i):
    if int(ct[i]) == 1:
        return 0.5
    if int(ct[i]) == 0:
        return -0.5
    


H = w(ct,0,1) *qml.PauliZ(0) @ qml.PauliZ(1)
+w(ct,0,6) *qml.PauliZ(0) @ qml.PauliZ(6) 
+w(ct,0,7) *qml.PauliZ(0) @ qml.PauliZ(7) 
+w(ct,1,3) *qml.PauliZ(1) @ qml.PauliZ(3)
+w(ct,1,7) *qml.PauliZ(1) @ qml.PauliZ(7) 
+w(ct,2,4) *qml.PauliZ(2) @ qml.PauliZ(4)
+w(ct,2,5) *qml.PauliZ(2) @ qml.PauliZ(5) 
+w(ct,2,7) *qml.PauliZ(2) @ qml.PauliZ(7) 
+w(ct,3,4) *qml.PauliZ(3) @ qml.PauliZ(4)
+w(ct,3,6) *qml.PauliZ(3) @ qml.PauliZ(6) 
+w(ct,4,5) *qml.PauliZ(4) @ qml.PauliZ(5)
+w(ct,5,6) *qml.PauliZ(5) @ qml.PauliZ(6)
+t(ct, 0) *qml.PauliZ(0)
+t(ct, 1) *qml.PauliZ(1)
+t(ct, 2) *qml.PauliZ(2)
+t(ct, 3) *qml.PauliZ(3)
+t(ct, 4) *qml.PauliZ(4)
+t(ct, 5) *qml.PauliZ(5)
+t(ct, 6) *qml.PauliZ(6)
+t(ct, 7) *qml.PauliZ(7)


  (-0.5) [Z7]


In [27]:

#Step 3: Ansatz
k=10
t=8
n=18
dev = qml.device("default.qubit", wires=n)


rng_seed = 0
np.random.seed(rng_seed)
weights = np.random.randn(k, requires_grad=True)


def circuit(w):
    
    #encode plaintext
    for i in range(k):
        qml.Identity(wires=i)
    
    for i, bit in enumerate(pt):
        if bit =="1":
            qml.PauliX(wires=i+k)
        else:
            qml.Identity(wires=i+k)
    
    #Ansatz
    
    for i in range(k):
        qml.Hadamard(wires=i)
    
       
    for idx, element in enumerate(w):
        qml.RY(element, wires=idx)
    
    
    
    for i in range(k-1):
       qml.CZ(wires=[i, i+1])
    
drawer = qml.draw(circuit)

print(qml.draw(circuit)(weights))

 0: ──I──H──RY(1.76)──╭●─────────────────────────┤  
 1: ──I──H──RY(0.40)──╰Z─╭●──────────────────────┤  
 2: ──I──H──RY(0.98)─────╰Z─╭●───────────────────┤  
 3: ──I──H──RY(2.24)────────╰Z─╭●────────────────┤  
 4: ──I──H──RY(1.87)───────────╰Z─╭●─────────────┤  
 5: ──I──H──RY(-0.98)─────────────╰Z─╭●──────────┤  
 6: ──I──H──RY(0.95)─────────────────╰Z─╭●───────┤  
 7: ──I──H──RY(-0.15)───────────────────╰Z─╭●────┤  
 8: ──I──H──RY(-0.10)──────────────────────╰Z─╭●─┤  
 9: ──I──H──RY(0.41)──────────────────────────╰Z─┤  
10: ──X──────────────────────────────────────────┤  
11: ──X──────────────────────────────────────────┤  
12: ──X──────────────────────────────────────────┤  
13: ──X──────────────────────────────────────────┤  
14: ──X──────────────────────────────────────────┤  
15: ──X──────────────────────────────────────────┤  
16: ──X──────────────────────────────────────────┤  
17: ──X──────────────────────────────────────────┤  


In [26]:
#SDES
dev = qml.device("default.qubit", wires=18)


@qml.qnode(dev) #do i need this here
def sdes_full(w): 

    circuit(w)

    #algo
    qml.CNOT(wires=[0,16])
    qml.CNOT(wires=[6,13])
    qml.CNOT(wires=[8,17])
    qml.CNOT(wires=[3,14])
    qml.CNOT(wires=[14,10])
    qml.MultiControlledX(control_wires=[13,14,16,17], wires=10)
    qml.MultiControlledX(control_wires=[16,17], wires=10)
    qml.PauliX(wires=[16])
    qml.MultiControlledX(control_wires=[13,16], wires=10)
    qml.PauliX(wires=[17])
    qml.MultiControlledX(control_wires=[16,17], wires=11)
    qml.PauliX(wires=[16])
    qml.MultiControlledX(control_wires=[13,14,16,17], wires=11)
    qml.PauliX(wires=[13])
    qml.MultiControlledX(control_wires=[13,16], wires=11)
    qml.PauliX(wires=[14])
    qml.MultiControlledX(control_wires=[14,16], wires=11)
    qml.PauliX(wires=[13])
    qml.PauliX(wires=[14])
    qml.PauliX(wires=[17])
    qml.CNOT(wires=[0,16])
    qml.CNOT(wires=[6,13])
    qml.CNOT(wires=[8,17])
    qml.CNOT(wires=[3,14])
    qml.CNOT(wires=[7,17])
    qml.CNOT(wires=[2,14])
    qml.CNOT(wires=[9,16])
    qml.CNOT(wires=[5,13])
    #s_1box
    qml.CNOT(wires=[14,12])
    qml.MultiControlledX(control_wires=[13,16,17], wires=15)
    qml.PauliX(wires=[13])
    qml.MultiControlledX(control_wires=[13,17], wires=15)
    qml.MultiControlledX(control_wires=[13,17], wires=12)
    qml.PauliX(wires=[14])
    qml.MultiControlledX(control_wires=[13,14,16,17], wires=12)
    qml.MultiControlledX(control_wires=[13,16], wires=15)
    qml.PauliX(wires=[13])
    qml.PauliX(wires=[16])
    qml.MultiControlledX(control_wires=[13,16], wires=12)
    qml.PauliX(wires=[14])
    qml.PauliX(wires=[17])
    qml.MultiControlledX(control_wires=[13,14,17], wires=15)
    qml.PauliX(wires=[16])
    qml.PauliX(wires=[17])
    qml.CNOT(wires=[7,17])
    qml.CNOT(wires=[2,14])
    qml.CNOT(wires=[9,16])
    qml.CNOT(wires=[5,13])
    qml.CNOT(wires=[7,10])
    qml.CNOT(wires=[2,11])
    qml.CNOT(wires=[5,15])
    qml.CNOT(wires=[4,12])
    #s_0box
    qml.CNOT(wires=[12,16])
    qml.MultiControlledX(control_wires=[10,11,12,15], wires=16)
    qml.MultiControlledX(control_wires=[10,15], wires=16)
    qml.PauliX(wires=[10])
    qml.MultiControlledX(control_wires=[10,11], wires=16)
    qml.PauliX(wires=[15])

    qml.MultiControlledX(control_wires=[10,15], wires=13)
    qml.PauliX(wires=[10])
    qml.MultiControlledX(control_wires=[10,11,12,15], wires=13)
    qml.PauliX(wires=[11])
    qml.MultiControlledX(control_wires=[10,11], wires=13)
    qml.PauliX(wires=[12])
    qml.MultiControlledX(control_wires=[10,12], wires=13)
    qml.PauliX(wires=[11])
    qml.PauliX(wires=[12])
    qml.PauliX(wires=[15])

    qml.CNOT(wires=[7,10])
    qml.CNOT(wires=[2,11])
    qml.CNOT(wires=[5,15])
    qml.CNOT(wires=[4,12])
    qml.CNOT(wires=[9,15])
    qml.CNOT(wires=[1,12])
    qml.CNOT(wires=[8,10])
    qml.CNOT(wires=[0,11])

    #s1_box
    qml.CNOT(wires=[12,14])
    qml.MultiControlledX(control_wires=[10,11,15], wires=17)
    qml.PauliX(wires=[11])
    qml.MultiControlledX(control_wires=[11,15], wires=17)
    qml.MultiControlledX(control_wires=[11,15], wires=14)
    qml.PauliX(wires=[12])
    qml.MultiControlledX(control_wires=[10,11,12,15], wires=14)
    qml.MultiControlledX(control_wires=[10,11], wires=17)
    qml.PauliX(wires=[10])
    qml.PauliX(wires=[11])
    qml.MultiControlledX(control_wires=[10,11], wires=14)
    qml.PauliX(wires=[12])
    qml.PauliX(wires=[15])
    qml.MultiControlledX(control_wires=[11,12,15], wires=17)
    qml.PauliX(wires=[10])
    qml.PauliX(wires=[15])
    qml.CNOT(wires=[9,15])
    qml.CNOT(wires=[1,12])
    qml.CNOT(wires=[8,10])
    qml.CNOT(wires=[0,11])
    qml.SWAP(wires=[10,16])
    qml.SWAP(wires=[11,13])
    qml.SWAP(wires=[12,14])
    qml.SWAP(wires=[15,17])

    #return [qml.expval(qml.PauliZ(i)) for i in range(10, 18)]

In [28]:
#Cost function
@qml.qnode(dev, interface="autograd")
def cost_fn(theta):
    sdes_full(theta)
    return qml.expval(H)


In [32]:
lr =  0.94
steps = 30
opt = qml.GradientDescentOptimizer(stepsize=steps)
theta = np.zeros(8, requires_grad=True)

In [33]:
for n in range(steps):
    theta, prev_energy = opt.step_and_cost(cost_fn, theta)
    samples = cost_fn(theta)

energy_VQE = cost_fn(theta)
theta_opt = theta

TypeError: sdes_full() takes 0 positional arguments but 1 was given

In [ ]:
#to do:
# fix hamiltonian to act on the last 8
#find why it isnt giving me the right answers 